<a href="https://colab.research.google.com/github/OdysseusPolymetis/initiation_ia/blob/main/video_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q diffusers transformers accelerate
!pip install -q imageio imageio-ffmpeg opencv-python

In [ ]:
import torch
import numpy as np
import cv2
import imageio
from google.colab import files
from IPython.display import HTML, display
import matplotlib.pyplot as plt
import os
from base64 import b64encode
from diffusers import TextToVideoZeroPipeline

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Utilisation de: {device}")

Utilisation de: cuda


In [ ]:
model_id = "runwayml/stable-diffusion-v1-5"
pipe = TextToVideoZeroPipeline.from_pretrained(
    model_id,
    torch_dtype=torch.float16 if device == "cuda" else torch.float32
).to(device)

In [ ]:
# Configuration standard plus longue
standard_long = {
    'video_length': 24,  # Vidéo 3x plus longue que le défaut
    'motion_x': 12,
    'motion_y': 12,
    't0': 45,
    't1': 48,
    'fps': 8
}

# Configuration très longue (attention: peut nécessiter plus de mémoire GPU)
tres_long = {
    'video_length': 36,  # Vidéo 4x plus longue que le défaut
    'motion_x': 12,
    'motion_y': 12,
    't0': 45,
    't1': 48,
    'fps': 40
}

# Configuration longue avec mouvement lent
long_mouvement_lent = {
    'video_length': 24,
    'motion_x': 8,    # Mouvement plus léger
    'motion_y': 8,
    't0': 45,
    't1': 48,
    'fps': 6          # Lecture plus lente
}

# Configuration longue avec mouvement rapide
long_mouvement_rapide = {
    'video_length': 24,
    'motion_x': 16,   # Mouvement plus prononcé
    'motion_y': 16,
    't0': 45,
    't1': 48,
    'fps': 10         # Lecture plus rapide
}

In [ ]:
def generer_video_longue(prompt,
                       video_length=24,     # Vidéo plus longue (24 frames par défaut)
                       motion_x=12,         # Force du mouvement horizontal
                       motion_y=12,         # Force du mouvement vertical
                       t0=45,               # Paramètre T
                       t1=48,               # Paramètre T'
                       num_inference_steps=50, # Étapes d'inférence
                       fps=8,               # Images par seconde
                       output_name="video_longue.mp4"):

    print(f"Génération de vidéo longue pour: '{prompt}'")
    print(f"Paramètres: video_length={video_length}, motion_x={motion_x}, motion_y={motion_y}, fps={fps}")
    print("Cela peut prendre quelques minutes...")

    # Génération de la vidéo avec les paramètres spécifiés
    result = pipe(
        prompt=prompt,
        video_length=video_length,      # Nombre de frames augmenté
        motion_field_strength_x=motion_x,
        motion_field_strength_y=motion_y,
        t0=t0,
        t1=t1,
        num_inference_steps=num_inference_steps
    ).images

    # Conversion et sauvegarde selon la documentation
    result = [(r * 255).astype("uint8") for r in result]
    imageio.mimsave(output_name, result, fps=fps)

    # Affichage de la vidéo dans le notebook
    mp4 = open(output_name, 'rb').read()
    data_url = f"data:video/mp4;base64,{b64encode(mp4).decode()}"
    display(HTML(f"""
    <video width=512 controls>
        <source src="{data_url}" type="video/mp4">
    </video>
    """))

    # Lien de téléchargement
    files.download(output_name)
    print(f"Vidéo générée avec {video_length} frames")
    print(f"Durée: {video_length/fps:.2f} secondes à {fps} FPS")
    print(f"Sauvegardée sous '{output_name}' et disponible au téléchargement")


In [ ]:
prompt_litterature = "Emmanuel Macron dansant le tango devant une foule de gens"

In [ ]:
generer_video_longue(prompt_litterature, **tres_long)